In [1]:
import os
import sys
import torch
import numpy as np
import argparse
import torchvision.transforms as T
from data_utils import *

from torch.utils.data import DataLoader, Dataset
import pickle
import PIL
os.chdir("..")
from src.utils import *

In [2]:
for subject in [1,2,5,7]:
    # create_whole_region_imagery_unnormalized(subject, mask=False)
    create_whole_region_imagery_normalized(subject, mask=False)

tensor(3.7253e-09) tensor(1.0000)
tensor(3.4769e-08) tensor(1.0000)
tensor(4.9671e-08) tensor(1.0000)
tensor(1.1797e-08) tensor(1.0000)
tensor(-3.7253e-09) tensor(1.0000)
tensor(-6.2088e-09) tensor(1.0000)


KeyboardInterrupt: 

In [ ]:
def zscore(x, mean=None, stddev=None, return_stats=False):
    new_x = x
    if mean is not None:
        m = mean
    else:
        m = torch.mean(new_x, axis=0, keepdims=True)
    if stddev is not None:
        s = stddev
    else:
        s = torch.std(new_x, axis=0, keepdims=True)
    if return_stats:
        return (x - m)/(s+1e-6), m, s
    else:
        x = torch.where(s==0, (new_x - m), (new_x - m)/s)
        return x

In [ ]:
# nsd_root = '/export/raid1/home/surly/raid1/kendrick-data/nsd/'
# stim_dir = nsd_root + 'nsddata_stimuli/stimuli/nsd/'
# beta_dir = nsd_root + 'nsddata_betas/ppdata/'
# mask_dir= nsd_root + 'nsddata/ppdata/'
# img_stim_file = stim_dir + "nsdimagery_stimuli.pkl3"
# beta_subj = beta_dir + "subj%02d/func1pt8mm/nsdimagerybetas_fithrf/betas_nsdimagery.nii.gz"%subj

def image_feature_fn(image):
    '''take uint8 image and return floating point (0,1), either color or bw'''
    return image.astype(np.float32) / 255

## LOAD THE STIM IMAGES AND SEQUENCE ALIGNMENT DESCRIPTORS
stim_dir = "data/nsddata_stimuli/stimuli/nsd/"
img_stim_file = stim_dir + "nsdimagery_stimuli.pkl3"
ex_file = open(img_stim_file, 'rb')
imagery_dict = pickle.load(ex_file)
print(imagery_dict.keys())
ex_file.close()
exps = imagery_dict['exps']
cues = imagery_dict['cues']
image_map  = imagery_dict['image_map']
image_data = imagery_dict['image_data']
# for i, im in enumerate(image_data):
#     image = Image.fromarray(im.transpose(1,2,0))
#     image.save("data/nsddata_stimuli/stimuli/imagery_images/{}.png".format(i))

dict_keys(['meta_dm', 'gdm', 'gcm', 'image_data', 'image_map', 'exps', 'cues', 'modes'])


In [ ]:
def zscore(x, mean=None, stddev=None, return_stats=False):
    if mean is not None:
        m = mean
    else:
        m = np.mean(x, axis=0, keepdims=True)
    if stddev is not None:
        s = stddev
    else:
        s = np.std(x, axis=0, keepdims=True)
    if return_stats:
        return (x - m)/(s+1e-6), m, s
    else:
        return (x - m)/(s+1e-6)


## EXAMPLE CONDITION AVERAGED RESPONSES
def condition_average(data, cond):
    idx, idx_count = np.unique(cond, return_counts=True)
    idx_list = [cond==i for i in np.sort(idx)]
    avg_data = np.zeros(shape=(len(idx),)+data.shape[1:], dtype=np.float32)
    for i,m in enumerate(idx_list):
        avg_data[i] = np.mean(data[m], axis=0)
    return avg_data

meta_cond_idx = {
    'visA': np.arange(len(exps))[exps=='visA'],
    'visB': np.arange(len(exps))[exps=='visB'],
    'imgA_1': np.arange(len(exps))[exps=='imgA_1'],
    'imgA_2': np.arange(len(exps))[exps=='imgA_2'],
    'imgB_1': np.arange(len(exps))[exps=='imgB_1'],
    'imgB_2': np.arange(len(exps))[exps=='imgB_2']
}

cond_idx = {
    'visA': np.arange(len(exps))[exps=='visA'],
    'visB': np.arange(len(exps))[exps=='visB'],
    'imgA': np.arange(len(exps))[np.logical_or(exps=='imgA_1', exps=='imgA_2')],
    'imgB': np.arange(len(exps))[np.logical_or(exps=='imgB_1', exps=='imgB_2')]
}


In [ ]:
## LOAD THE DATA WITHOUT GLOBAL ZSCORING (EVEN IF IT IS ZSCORED WE ARE GONNA RE-ZSCORE LATER)

# voxel_data_raw = {}
# subjects = [1,2,3,4,5,6,7,8]
# for k,s in enumerate(subjects):
#     # print ('--------  subject %d  -------' % s)
#     create_whole_region_imagery_unnormalized(s)
#     create_whole_region_imagery_normalized(s)
    # voxel_data_raw[s] = torch.load("data/preprocessed_data/subject{}/nsd_imagery_unnormalized.pt".format(s))
    # print (voxel_data_raw[s].shape)


# ## NORMALIZATION OF THE DATA FOR EACH INDIVIDUAL TRIAL
# voxel_data_n = {}
# for s in voxel_data_raw.keys():
#     voxel_data_n[s] = np.copy(voxel_data_raw[s])
#     for c,idx in meta_cond_idx.items():
#         voxel_data_n[s][idx] = zscore(voxel_data_raw[s][idx])
#     print(voxel_data_n[s])


# cond_im_idx = {n: [image_map[c] for c in cues[idx]] for n,idx in cond_idx.items()}
# ## EXAMPLE USE
# for c, idx, im_idx in zip_dict(cond_idx, cond_im_idx): # loop conditions
#     data_single = voxel_data_n[s][idx]
#     data = condition_average(data_single, im_idx)
#     print(data.shape)

In [ ]:
x, y = load_nsd_mental_imagery(vector = "c", subject=1, mode="imagery", stimtype="all", average=True, nest=False)
x, y = load_nsd_mental_imagery(vector = "images", subject=1, mode="imagery", stimtype="simple", average=True, nest=False)
x, y = load_nsd_mental_imagery(vector = "z_vdvae", subject=1, mode="imagery", stimtype="complex", average=True, nest=False)


torch.Size([12, 15724]) torch.Size([12, 1024])
torch.Size([6, 15724]) torch.Size([6, 1024])
torch.Size([6, 15724]) torch.Size([6, 1024])
